In [2]:
import os
from configparser import ConfigParser
path_config_file = os.getcwd().split('docrv2_sroie')[0] + 'docrv2_sroie/' + 'config.ini'
common_variable = ConfigParser()
common_variable.read(path_config_file)
print(common_variable.sections())
print(common_variable['database']['module_path'])

from PIL import Image, ImageDraw, ImageFont, ImageTransform
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
from glob import glob
import sys
sys.path.append(common_variable['database']['module_path'])
from db_util import *
from db import *
from random import choice

['database', 'mlflow', 'font']
/home/dk/docrv2_sroie/modules/database/


In [2]:
# set simulator data
fontdir = '../../../data/font/*.ttf'
fontpath = glob(fontdir)
textdata = pd.read_csv('../../../data/textdata.csv',sep=',',header=None)
textdata = list(textdata.values)

In [3]:
# set simulator parameter
imageSize = (256*4, 128)
fontsSize = (20,30)

In [4]:
# some generators
def fontGenerator(fontpath, fontsSize):
    while True:
        fontselection = np.random.randint(0,len(fontpath),1)[0]
        fontsize = np.random.randint(fontsSize[0],fontsSize[1],1)[0]
        yield ImageFont.truetype(fontpath[fontselection], fontsize)
fontgen = fontGenerator(fontpath, fontsSize)

def randcapitalize(text):
    return ''.join(choice((str.upper, str.lower))(c) for c in text)

def textGenerator(textdata):
    while True:
        textselection = np.random.randint(0,len(textdata),1)[0]
        randchoice = np.random.randint(0,10,1)[0]
        if randchoice <= 2:
            yield str(textdata[textselection]) 
        elif randchoice > 2 and randchoice < 8:
            yield str(textdata[textselection]).lower()
        else:
            yield randcapitalize(str(textdata[textselection]))
textgen = textGenerator(textdata[0])

def locGenerator(imageSize):
    while True:
        x = np.random.randint(imageSize[0]//8,imageSize[0]//4,1)[0]
        y = np.random.randint(imageSize[1]//8,imageSize[1]//4,1)[0]
        yield [x,y]
locgen = locGenerator(imageSize)

def textcoordinates(font, text, loc):
    char_table = pd.DataFrame({'x1':[], 'y1':[], 'x2':[], 'y2':[], 'x3':[], 'y3':[],
                                     'x4':[], 'y4':[], 'char':[]})
    for i, char in enumerate(text):
        b1 = font.getsize('A')[1]
        r, b2 = font.getsize(text[:i+1])
        b = b1 if b1 < b2 else b2
        w, _ = font.getmask(char).size
        _, h = font.getmask('A').size
        r += loc[0]
        b += loc[1]
        t = b - h
        l = r - w
        right = int(np.round(r))
        bottom = int(np.round(b))
        top = int(np.round(t))
        left = int(np.round(l))
        if i == 0:
            char_data = pd.DataFrame({'x1':[left-2], 'y1':[top], 
                                     'x2':[left-2], 'y2':[top],
                                     'x3':[left-2], 'y3':[bottom],
                                     'x4':[left-2], 'y4':[bottom], 'char':' '})
            char_table = char_table.append(char_data, ignore_index=True)
        if text[i] != ' ':
            char_data = pd.DataFrame({'x1':[left], 'y1':[top], 
                                     'x2':[right], 'y2':[top],
                                     'x3':[right], 'y3':[bottom],
                                     'x4':[left], 'y4':[bottom], 'char':text[i]})
            char_table = char_table.append(char_data, ignore_index=True)
        if i == len(text)-1:
            char_data = pd.DataFrame({'x1':[right+2], 'y1':[top], 
                                     'x2':[right+2], 'y2':[top],
                                     'x3':[right+2], 'y3':[bottom],
                                     'x4':[left+2], 'y4':[bottom], 'char':' '})
            char_table = char_table.append(char_data, ignore_index=True)
        char_table.x1 = char_table.x1.astype(int)
        char_table.y1 = char_table.y1.astype(int)
        char_table.x2 = char_table.x2.astype(int)
        char_table.y2 = char_table.y2.astype(int)
        char_table.x3 = char_table.x3.astype(int)
        char_table.y3 = char_table.y3.astype(int)
        char_table.x4 = char_table.x4.astype(int)
        char_table.y4 = char_table.y4.astype(int)
    return char_table


def create_image():
    # create image
    img = Image.new('RGB', imageSize, color='White')
    draw = ImageDraw.Draw(img)
    
    # first text
    font = next(fontgen)
    text = next(textgen)
    loc = [10, 64]
    draw.text(loc, text, fill=(0,0,0), font=font)
    tb = textcoordinates(font, text, loc)
    cr = [min(tb['x1']),min(tb['y1'])]-np.random.randint(2,6,2)
    cr = np.append(cr, [max(tb['x3']),max(tb['y3'])]+np.random.randint(2,6,2))
    img = img.crop(cr)
    return img, text

In [5]:
path = '/home/dk/docr2/DB/'
name = 'recog_test_1000'
label = 'recog'
cache = {}
numImage = 1000
db_data = create_data_db(path=path, name=name, description='datadb_recog',data_class=label)
db_data_db = db_data.open_db(b'db_data')
for idx in tqdm(range(numImage)):
    img, txt = create_image()
    cache = update_cache(cache, index=idx, label=label, ref=label, text=txt, image=img)
    if idx % 5000 == 0:
        write_cache_to_db(db_data, db_data_db, cache)
        cache = {}
        
write_cache_to_db(db_data, db_data_db, cache)
cache = {}

100%|██████████| 1000/1000 [00:13<00:00, 74.78it/s]


In [6]:
path = '/home/dk/docr2/DB/'
name = 'recog_train_100000'
label = 'recog'
cache = {}
numImage = 100000
db_data = create_data_db(path=path, name=name, description='datadb_recog',data_class=label)
db_data_db = db_data.open_db(b'db_data')
for idx in tqdm(range(numImage)):
    img, txt = create_image()
    cache = update_cache(cache, index=idx, label=label, ref=label, text=txt, image=img)
    if idx % 5000 == 0:
        write_cache_to_db(db_data, db_data_db, cache)
        cache = {}
        
write_cache_to_db(db_data, db_data_db, cache)
cache = {}

100%|██████████| 100000/100000 [22:16<00:00, 74.81it/s]


In [4]:
refresh_main_db(force_path=True)

+--------------------+---------------------------------------------+
| Key                | Value                                       |
+--------------------+---------------------------------------------+
| d2gan_test_1       | /home/dk/docrv2_sroie/DB/d2gan_test_1       |
| d2gan_test_100     | /home/dk/docrv2_sroie/DB/d2gan_test_100     |
| d2gan_test_1000    | /home/dk/docrv2_sroie/DB/d2gan_test_1000    |
| d2gan_train_100000 | /home/dk/docrv2_sroie/DB/d2gan_train_100000 |
| recog_test_1       | /home/dk/docrv2_sroie/DB/recog_test_1       |
| recog_test_1000    | /home/dk/docrv2_sroie/DB/recog_test_1000    |
| recog_train_100000 | /home/dk/docrv2_sroie/DB/recog_train_100000 |
+--------------------+---------------------------------------------+


['d2gan_train_100000',
 'd2gan_test_1000',
 'recog_train_100000',
 'recog_test_1000',
 'd2gan_test_100',
 'recog_test_1',
 'd2gan_test_1']

In [18]:
temptext = ''
plen = 0
for tt in textdata[0]:
    if len(str(tt))>plen:
        plen = len(tt)
print(plen)

69
